In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [4]:
def get_shape(tensor):
    "return the shape of tensor as list"
    return tensor.get_shape().as_list()

def print_var(string):
    "print var_name and var_shape in collection named string"
    print("\n".join(["{} : {}".format(v.name, get_shape(v)) for v in tf.get_collection(string)]))
    
def clip(x, vmax = 1-1e-7, vmin = 1e-7):
    "clip the x between vmax, vmin"
    return tf.clip_by_value(x, clip_value_max=vmax, clip_value_min=vmin)

In [5]:
mnist = input_data.read_data_sets('../MNIST_data', one_hot=True)

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
train = {}
test = {}
val = {}
train['input'] = mnist.train.images
train['output'] = mnist.train.labels
test['input'] = mnist.test.images
test['output'] = mnist.test.labels
val['input'] = mnist.validation.images
val['output'] = mnist.validation.labels

In [7]:
# Model parameter
image_size = 28*28
image_row = 28
conv_filter1 = [5, 5]
conv_channel1 = 30
conv_filter2 = [3, 3]
conv_channel2 = 50
fc_hidden = 500
nclasses = 10

# Train parameter
learning_rate = 1e-4
epoch = 5
ntrain = len(train['input'])
batch_size = 100
nbatch = ntrain//batch_size
log_every = 50

In [8]:
x = tf.placeholder(dtype = tf.float32, shape = [None, image_size], name = 'image')
y = tf.placeholder(dtype = tf.float32, shape = [None, nclasses], name = 'label')

x_reshape = tf.reshape(x, [-1, image_row, image_row, 1]) # 4D 

conv_layer1 = {}
conv_layer2 = {}
fc_layer1 = {}
fc_layer2 = {}

with tf.variable_scope("conv_layer1"):
    conv_layer1['w'] = tf.Variable(initial_value = tf.random_normal(shape = [conv_filter1[0], conv_filter1[1], 1, conv_channel1], mean = 0.0, stddev = 0.1),
                              name = 'weight')
    conv_layer1['b'] = tf.Variable(initial_value = tf.constant(value=0.01, shape = [conv_channel1]),
                              name = 'bias')
    conv_layer1['conv'] = tf.nn.conv2d(x_reshape, conv_layer1['w'], strides = [1, 1, 1, 1], padding = "VALID") + conv_layer1['b']
    conv_layer1['relu'] = tf.nn.relu(conv_layer1['conv'])
    conv_layer1['pool'] = tf.nn.max_pool(conv_layer1['relu'], ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'VALID')

with tf.variable_scope("conv_layer2"):
    conv_layer2['w'] = tf.Variable(initial_value = tf.random_normal(shape = [conv_filter2[0], conv_filter2[1], conv_channel1, conv_channel2], mean = 0.0, stddev = 0.1),
                              name = 'weight')
    conv_layer2['b'] = tf.Variable(initial_value = tf.constant(value=0.01, shape = [conv_channel2]),
                              name = 'bias')
    conv_layer2['conv'] = tf.nn.conv2d(conv_layer1['pool'], conv_layer2['w'], strides = [1, 1, 1, 1], padding = "VALID") + conv_layer2['b']
    conv_layer2['relu'] = tf.nn.relu(conv_layer2['conv'])
    conv_layer2['pool'] = tf.nn.max_pool(conv_layer2['relu'], ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'VALID')
    conv_shape = get_shape(conv_layer2['pool'])
    flat_size = conv_shape[1]*conv_shape[2]*conv_shape[3]
    conv_layer2['flat'] = tf.reshape(conv_layer2['pool'], [-1, flat_size])    
    
with tf.variable_scope("fc_layer1"):
    fc_layer1['w'] = tf.Variable(initial_value = tf.random_normal(shape = [flat_size, fc_hidden], mean = 0.0, stddev = 0.1),
                              name = 'weight')
    fc_layer1['b'] = tf.Variable(initial_value = tf.constant(value=0.01, shape = [fc_hidden]),
                              name = 'bias')
    fc_layer1['output'] = tf.nn.relu(tf.add(tf.matmul(conv_layer2['flat'], fc_layer1['w']), fc_layer1['b']))    

with tf.variable_scope("fc_layer2"):
    fc_layer2['w'] = tf.Variable(initial_value = tf.random_normal(shape = [fc_hidden, nclasses], mean = 0.0, stddev = 0.1),
                              name = 'weight')
    fc_layer2['b'] = tf.Variable(initial_value = tf.constant(value=0.01, shape = [nclasses]),
                              name = 'bias')
    fc_layer2['output'] = tf.nn.softmax(tf.add(tf.matmul(fc_layer1['output'], fc_layer2['w']), fc_layer2['b']))    
    
print("Reshape {} -> {}".format(get_shape(x), get_shape(x_reshape)))
print("Convolution layer 1")
print("Conv {} -> {}".format(get_shape(x_reshape), get_shape(conv_layer1['conv'])))
print("Pool {} -> {}".format(get_shape(conv_layer1['conv']), get_shape(conv_layer1['pool'])))    
print("Convolution layer 2")
print("Conv {} -> {}".format(get_shape(conv_layer1['pool']), get_shape(conv_layer2['conv'])))
print("Pool {} -> {}".format(get_shape(conv_layer2['conv']), get_shape(conv_layer2['pool'])))   
print("Flat {} -> {}".format(get_shape(conv_layer2['pool']), get_shape(conv_layer2['flat'])))
print("Fully connected layer 1")
print("Linear {} -> {}".format(get_shape(conv_layer2['flat']), get_shape(fc_layer1['output'])))
print("Fully connected layer 2")
print("Linear {} -> {}".format(get_shape(fc_layer1['output']), get_shape(fc_layer2['output'])))

Reshape [None, 784] -> [None, 28, 28, 1]
Convolution layer 1
Conv [None, 28, 28, 1] -> [None, 24, 24, 30]
Pool [None, 24, 24, 30] -> [None, 12, 12, 30]
Convolution layer 2
Conv [None, 12, 12, 30] -> [None, 10, 10, 50]
Pool [None, 10, 10, 50] -> [None, 5, 5, 50]
Flat [None, 5, 5, 50] -> [None, 1250]
Fully connected layer 1
Linear [None, 1250] -> [None, 500]
Fully connected layer 2
Linear [None, 500] -> [None, 10]


In [9]:
print_var("trainable_variables")

conv_layer1/weight:0 : [5, 5, 1, 30]
conv_layer1/bias:0 : [30]
conv_layer2/weight:0 : [3, 3, 30, 50]
conv_layer2/bias:0 : [50]
fc_layer1/weight:0 : [1250, 500]
fc_layer1/bias:0 : [500]
fc_layer2/weight:0 : [500, 10]
fc_layer2/bias:0 : [10]


In [10]:
cross_entropy = -tf.reduce_mean(y*tf.log(clip(fc_layer2['output'])))

correct_prediction = tf.equal(tf.argmax(fc_layer2['output'], 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

run_train = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [14]:
train_accuracy_tract = []
val_accuracy_tract = []

for epoch_ in range(epoch):
    index = np.arange(len(train['input']))
    np.random.shuffle(index)
    trX = train['input'][index]
    trY = train['output'][index]
    val_feed_dict = {x : val['input'], 
                     y : val['output']}
    
    for nbatch_ in range(nbatch):
        train_feed_dict = {x : trX[batch_size*nbatch_:batch_size*(nbatch_+1)], 
                           y : trY[batch_size*nbatch_:batch_size*(nbatch_+1)]}
        sess.run(run_train, feed_dict=train_feed_dict)
        
        if nbatch_%log_every == log_every - 1:
            train_accuracy = 100*sess.run(accuracy, feed_dict=train_feed_dict)
            val_accuracy = 100*sess.run(accuracy, feed_dict= val_feed_dict)
            train_accuracy_tract.append(train_accuracy)
            val_accuracy_tract.append(val_accuracy)
            print("Epoch(%d/%d) %d th batch train : %.1f %%, validation : %.1f %%"%(epoch_+1, epoch, nbatch_+1, train_accuracy, val_accuracy))

test_feed_dict = {x : test['input'],  
                  y : test['output']}

test_accuracy = 100*sess.run(accuracy, feed_dict=test_feed_dict)
print("Test accuracy=%.1f %%"%(test_accuracy))



Epoch(1/5) 50 th batch train : 75.0 %, validation : 76.8 %
Epoch(1/5) 100 th batch train : 86.0 %, validation : 87.9 %
Epoch(1/5) 150 th batch train : 93.0 %, validation : 90.8 %
Epoch(1/5) 200 th batch train : 90.0 %, validation : 92.5 %
Epoch(1/5) 250 th batch train : 92.0 %, validation : 93.2 %
Epoch(1/5) 300 th batch train : 94.0 %, validation : 94.0 %
Epoch(1/5) 350 th batch train : 96.0 %, validation : 95.0 %
Epoch(1/5) 400 th batch train : 98.0 %, validation : 95.0 %
Epoch(1/5) 450 th batch train : 95.0 %, validation : 95.8 %
Epoch(1/5) 500 th batch train : 99.0 %, validation : 95.8 %
Epoch(1/5) 550 th batch train : 95.0 %, validation : 96.2 %
Epoch(2/5) 50 th batch train : 93.0 %, validation : 96.5 %
Epoch(2/5) 100 th batch train : 96.0 %, validation : 96.6 %
Epoch(2/5) 150 th batch train : 93.0 %, validation : 96.5 %
Epoch(2/5) 200 th batch train : 95.0 %, validation : 96.8 %
Epoch(2/5) 250 th batch train : 99.0 %, validation : 96.7 %
Epoch(2/5) 300 th batch train : 95.0 %, va